# NLP pour l’analyse de critiques de films

## Contexte du projet

En règle générale, le nombre d'avis sur un film peu être important et par conséquent le temps de lecture de chaque commentaire peut être une tâche lourde. Alors comment déterminer de manière rapide si un film a eu du succès auprès des spectateurs (ou pas) ? Dans ce contexte, l’idée du projet est d’utiliser des algorithmes d'apprentissage automatique pour la tâche d'analyse de sentiment des spectateurs via leur critique.

Tout d’abord, il sera question que récupérer les données directement du site d’Allociné. En d’autres termes, nous allons scraper les pages qui nous intéressent sur ce site à savoir les critiques des personnes pour le film Inception et Sonic 2.

En navigant sur la page des critiques, vous vous apercevrez que seules deux types d’information ici nous intéresse : la note du spectateur ainsi que son avis. Pourquoi la note ? Parce que nous allons entraîner un modèle de type supervisé et plus précisément un classifieur et donc la note va nous aider à récupérer la classe pour étiqueter le commentaire. Pour cela, nous considérerons qu’une note au-dessus de 3 est considérée comme satisfaisante. Sinon, l’avis est négatif. Ici, nous avons donc réduit le problème à une classification binaire.

## Etape 1 : Web Scraping des données d'avis de spectateurs

Le web scrapping à été réalisé dans le script 'web_scraping.py' et les résultats sont sous forme de dataframes exporté grâce à joblib dans le fichier 'dataframes'

## Etape 2 : Préparation des données

Ayant maintenant nos jeux de données, il faut les préparer afin de pouvoir modéliser notre analyse de sentiments. Pour cela nous allons faire appel à plusieurs techniques :

- Des expressions régulières pour retirer les bruits (ponctuation, etc.) des commentaires.
- Du NLP pour tokeniser et réduire le corpus de chaque commentaire (afin par exemple de ne garder que les mots importants via les stopwords)
  - Des sacs de mots afin de « transformer » nos mots en nombres qui pourront alors être exploités dans un algorithme de Machine learning
  
*Les commentaires seront filtrés à leur essentiel.*

### *Import des librairies :*

In [245]:
import joblib
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

### *Import des jeux de données :*

In [229]:
df_sonic = joblib.load('dataframes')['sonic2']
df_inception = joblib.load('dataframes')['inception']

df = pd.concat([df_sonic, df_inception])

In [230]:
df.head()

,Commentaires,Notes
0,\nune bonne suite pour les aventures de sonic ...,4.0
1,\nCette suite de sonic est incroyable !Le fan ...,5.0
2,\nTrès bon film ont retrouve ce qui fait l’esp...,4.5
3,"\nSuper film de ouf, le fait que Knuckles soit...",4.0
4,\nMalgré deux nouveaux personnages de l'univer...,3.0


On remarque de suite que les commentaires ne sont pas encore bien mis en forme, il faudra arranger cela par la suite.

In [231]:
df.isna().sum()

Commentaires    0
Notes           0
dtype: int64

Aucune valeur nulle.

In [232]:
df.shape

(7350, 2)

On a un total de 7350 commentaires dans notre dataset

### *Gestion du bruit des commentaires :*

On définit une fonction pour supprimer le bruit :
- Suppresion des caractères spéciaux
- Désaccentuation
- Tout passer en minuscule

In [233]:
def standardize_text(df, content_field):
    df[content_field] = df[content_field].str.replace(r"http\S+", "")
    df[content_field] = df[content_field].str.replace(r"http", "")
    df[content_field] = df[content_field].str.replace(r"@\S+", "")
    df[content_field] = df[content_field].str.replace(r"[0-9(),!?@\'\:\.\/\^\-\`\"\_\n]", " ")
    df[content_field] = df[content_field].str.replace(r"@", "at")
    df[content_field] = df[content_field].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[content_field] = df[content_field].str.lower()
    return df

In [234]:
standardize_text(df, 'Commentaires')
df.head()

/tmp/ipykernel_5550/1153927108.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"http\S+", "")
/tmp/ipykernel_5550/1153927108.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"@\S+", "")
/tmp/ipykernel_5550/1153927108.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df[content_field] = df[content_field].str.replace(r"[0-9(),!?@\'\:\.\/\^\-\`\"\_\n]", " ")


,Commentaires,Notes
0,une bonne suite pour les aventures de sonic a...,4.0
1,cette suite de sonic est incroyable le fan q...,5.0
2,tres bon film ont retrouve ce qui fait lespri...,4.5
3,super film de ouf le fait que knuckles soit ...,4.0
4,malgre deux nouveaux personnages de l univers...,3.0


### *Séparation du jeu de données :*

Les prochaines étapes de préparation ne doivent s'appliquer qu'au jeu d'entraînement, il nous faut garder un jeu de test intact.

In [235]:
y = df['Notes']
X = df.drop('Notes', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Notre jeu d'entrainement contient {X_train.shape[0]} commentaires et celui de test en contient {X_test.shape[0]}")

Notre jeu d'entrainement contient 5880 commentaires et celui de test en contient 1470


### *Tokenisation :*

Le but désormais est de constituer notre *corpus* (collection de commentaires) traité, c'est à dire sans *'stop words'* et *lemmatisé*.

- Les *'stop words'* sont les mots qui n'apportent aucune information.

- La *lematisation* consiste à réduire un mot à sa forme de base (ex : mangea devient manger). 

On utilisera la librarie `FrenchLefffLemmatizer` developpée par Claude Colombe

In [236]:
print(f"Voici la liste des stopwords de la langue française : {stopwords.words('french')}")

Voici la liste des stopwords de la langue française : ['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', '

On définit note fonction créer notre corpus :

In [237]:
def make_corpus(df, content_field):
    lemmatizer = FrenchLefffLemmatizer()
    corpus = []
    for i, row in df.iterrows():
        message = row.item()
        message = message.split()                                   
        message =[word for word in message if not word in set(stopwords.words('french'))]
        message = [lemmatizer.lemmatize(word) for word in message]
        message = ' '.join(message)
        corpus.append(message)
    return corpus

In [238]:
corpus = make_corpus(X_train, 'Commentaires') #Temps d'éxecution : ~30-45s

In [239]:
print(f"Quelques exemple de commentaires filtrés à l'essentiel : \n\n{corpus[456]}\n\n{corpus[1456]}\n\n{corpus[25]}\n\n{corpus[3569]}")

Quelques exemple de commentaires filtrés à l'essentiel : 

christopher nolan signe encore excellent film effet speciaux enormes bande aussi acteur parfaitement bien choisis pris dedans debut a fin minute ennuie merveilleux

thriller film action a tres haut niveau grand nolan scenario assez original faire film film complet excellent voit plus acteur jouer leurs personnage seul bemol quelques longueur idee tres develloppee quelques scenes rappelle shutter island scorcese aussi leonardo dicaprio etait aussi tres bon film allez voir regretterez

enorme scenario effet speciaux a voir absolument contre tentez distraire autre chose film pendant projection sou peine etre largue intrigue a sortie seance quel plaisir quelles impression beau cinema

rien compris desole vraiment rien si quelqu veut bien expliquer


Les commentaires sont donc maintenant bien traités, on peut passer à la prochaine étape.

## Etape 3 : Préparation des libellés

Jusque là, à chaque commentaire est associé une note de 1 à 5 et non une classe binaire. Il nous faut donc convertir nos notes en : 1 pour avis positif et 0 : pour avis négatif


In [240]:
y_train = y_train.astype('float')
y_test = y_test.astype('float')
print(type(y_train[0]), type(y_test[0]))

<class 'numpy.float64'> <class 'numpy.float64'>


In [241]:
def codage_libelle(data):
    """Convertit les notes en 0 ou 1 selon s'il s'agit d'un avis positif ou négatif"""
    positif = data >= 3
    negatif = data < 3
    data[positif] = 1
    data[negatif] = 0
    data = data.astype('int')
    return data

In [242]:
#y_train, y_test = codage_libelle(y_train), codage_libelle(y_test)

In [243]:
y_train.value_counts()

1    4971
0     909
Name: Notes, dtype: int64

On remarque tout de suite que notre dataset n'est pas équilibré, on a 5 fois plus d'avis positifs que négatifs. Pour y remédier on pourrait recourir au procédé de data augmentation.

## Etape 4 : Finalisation de nos jeux de données

Les données sont presque prêtes mais nos commentaires qui sont maintenant sous forme de sac de mots doivent être convertis en nombre. Pour cela, il va falloir vectoriser nos mots (technique des sacs de mots)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))